# 🚦 YOLO Detection Server V13 - MAX SPEED
### Optimized for 25+ FPS on P100 GPU
### Vehicle Detection Only + FP16 + imgsz=416

In [ ]:
# ═══════════════════════════════════════════════════════════
# Cell 1: CONFIGURATION - EDIT THIS!
# ═══════════════════════════════════════════════════════════

# HTTP API URL (for fetching cameras) - Port 3000
BACKEND_HTTP_URL = 'https://teens-relief-performed-blank.trycloudflare.com'

# WebSocket URL (for sending detection results) - Port 3001
BACKEND_WS_URL = 'wss://kevin-revolution-fish-foster.trycloudflare.com'

# Media Server URL (for FLV streams) - Port 8000
NMS_URL = 'https://designer-supply-later-landing.trycloudflare.com'

# Detection settings - SPEED OPTIMIZED
TRACK_LINE_Y = 50
CONFIDENCE = 0.4        # Lower for speed
IMGSZ = 416             # Smaller = faster (320, 416, 480, 640)
USE_HALF = True         # FP16 for speed
MAX_FPS = 30            # Target FPS (no throttling if 0)

# Only vehicle detection for speed (disable TL, LP, OCR)
ENABLE_TRAFFIC_LIGHT = False
ENABLE_LICENSE_PLATE = False

print(f"📡 Backend HTTP: {BACKEND_HTTP_URL}")
print(f"📡 Backend WS: {BACKEND_WS_URL}")
print(f"📺 NMS: {NMS_URL}")
print(f"⚡ IMGSZ: {IMGSZ} | HALF: {USE_HALF} | MAX_FPS: {MAX_FPS}")

In [ ]:
# Cell 2: Install Dependencies
!pip uninstall -y numpy pillow ultralytics > /dev/null 2>&1
!pip install "numpy<2.0.0" "pillow>=10.3.0" ultralytics opencv-python-headless requests websocket-client --upgrade --quiet
!wget -nc -q https://github.com/ultralytics/assets/releases/download/v8.3.0/yolo11n.pt

import numpy, PIL
print(f'✅ Deps: Numpy {numpy.__version__}, PIL {PIL.__version__}')

In [ ]:
# Cell 3: Load Model - SPEED OPTIMIZED
import warnings
warnings.filterwarnings('ignore')
import torch
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'🚀 Device: {device}')

# Use yolo11n (nano) for MAXIMUM speed, or yolo11s for balance
vehicle_model = YOLO('yolo11n.pt').to(device)
if USE_HALF and device == 'cuda':
    vehicle_model.model.half()  # FP16
    print('✅ Vehicle Model (yolo11n) - FP16 enabled')
else:
    print('✅ Vehicle Model (yolo11n)')

# Warmup
import numpy as np
dummy = np.zeros((480, 640, 3), dtype=np.uint8)
for _ in range(3):
    vehicle_model(dummy, imgsz=IMGSZ, verbose=False)
print('✅ Warmup complete')

In [ ]:
# Cell 4: Detection Function - SPEED OPTIMIZED
import cv2, numpy as np, time, threading
from datetime import datetime

VEHICLE_CLASSES = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']
camera_trackers = {}

def log(msg): print(f"[{datetime.now().strftime('%H:%M:%S')}] {msg}")

def detect_frame_fast(frame, camera_id):
    """Fast detection - vehicle only, no LP/OCR"""
    h, w = frame.shape[:2]
    result = {
        'camera_id': camera_id,
        'created_at': int(time.time() * 1000),
        'image_dimensions': {'width': w, 'height': h},
        'track_line_y': TRACK_LINE_Y
    }
    
    if camera_id not in camera_trackers:
        camera_trackers[camera_id] = {
            'tracks': {}, 'counted': {},
            'counts_up': {v:0 for v in VEHICLE_CLASSES},
            'counts_down': {v:0 for v in VEHICLE_CLASSES},
            'total_up': 0, 'total_down': 0
        }
    tr = camera_trackers[camera_id]
    
    t0 = time.time()
    dets, tracks, vcounts, crossings = [], {}, {v:0 for v in VEHICLE_CLASSES}, []
    ly = int(h * TRACK_LINE_Y / 100)
    
    # Fast inference with optimized settings
    for r in vehicle_model.track(
        frame, 
        persist=True, 
        verbose=False,
        imgsz=IMGSZ,
        conf=CONFIDENCE,
        half=USE_HALF
    ):
        for b in r.boxes:
            cls = vehicle_model.names[int(b.cls[0])]
            if cls not in VEHICLE_CLASSES: continue
            x1,y1,x2,y2 = map(int, b.xyxy[0])
            cx, cy = (x1+x2)//2, (y1+y2)//2
            det = {
                'class': cls, 'type': 'vehicle', 'confidence': float(b.conf[0]),
                'bbox': {'x1': x1/w, 'y1': y1/h, 'x2': x2/w, 'y2': y2/h},
                'bbox_pixels': [x1, y1, x2, y2]
            }
            if hasattr(b,'id') and b.id is not None:
                tid = int(b.id[0]); det['id'] = tid
                tracks[tid] = {'pos': (cx,cy), 'time': result['created_at'], 'class': cls}
            dets.append(det); vcounts[cls] += 1
    
    # Counting logic (simplified)
    for tid, info in tracks.items():
        if tid not in tr['tracks']: tr['tracks'][tid] = []
        if tr['tracks'][tid]:
            py, cy = tr['tracks'][tid][-1]['pos'][1], info['pos'][1]
            d = 1 if py <= ly < cy else (-1 if py >= ly > cy else 0)
            if d and f"{tid}_{d}" not in tr['counted']:
                tr['counted'][f"{tid}_{d}"] = True
                if d == 1: tr['counts_down'][info['class']] += 1; tr['total_down'] += 1
                else: tr['counts_up'][info['class']] += 1; tr['total_up'] += 1
                crossings.append({'id': tid, 'direction': d})
        tr['tracks'][tid].append({'pos': info['pos'], 'time': info['time'], 'class': info['class']})
        tr['tracks'][tid] = tr['tracks'][tid][-10:]  # Keep less history for speed
    
    result['vehicle'] = {
        'detections': dets, 
        'inference_time': (time.time()-t0)*1000,
        'vehicle_count': {
            'total_up': tr['total_up'], 'total_down': tr['total_down'],
            'by_type_up': tr['counts_up'].copy(), 'by_type_down': tr['counts_down'].copy(),
            'current': vcounts
        },
        'new_crossings': crossings
    }
    
    return result

print('✅ Fast detection ready')

In [ ]:
# Cell 5: WebSocket Client
import websocket
import json

class TrafficWebSocket:
    def __init__(self, url, camera_id, api_key):
        self.url = f"{url}?cameraId={camera_id}&apiKey={api_key}"
        self.camera_id = camera_id
        self.ws = None
        self.connected = False
        self.lock = threading.Lock()

    def connect(self):
        try:
            self.ws = websocket.WebSocketApp(
                self.url,
                on_open=lambda ws: setattr(self, 'connected', True),
                on_close=lambda ws,c,m: setattr(self, 'connected', False),
                on_error=lambda ws,e: None
            )
            threading.Thread(target=self.ws.run_forever, daemon=True).start()
            time.sleep(1)
            return self.connected
        except: return False

    def send(self, data):
        with self.lock:
            if self.connected and self.ws:
                try:
                    self.ws.send(json.dumps(data))
                    return True
                except: self.connected = False
            return False

print('✅ WebSocket ready')

In [ ]:
# Cell 6: Main Loop - MAX SPEED
import cv2, time, requests, threading
from collections import deque

camera_stats = {}
camera_ws = {}
camera_keys = {}

def fetch_cameras():
    try:
        resp = requests.get(f"{BACKEND_HTTP_URL}/api/camera/all", timeout=10)
        if resp.status_code == 200:
            data = resp.json()
            cams = data.get('metadata', data) if isinstance(data, dict) else data
            for cam in cams:
                if isinstance(cam, dict) and cam.get('_id'):
                    camera_keys[cam['_id']] = cam.get('camera_api_key', '')
            return list(camera_keys.keys())
    except Exception as e: log(f'❌ {e}')
    return []

def process_camera(camera_id):
    flv_url = f"{NMS_URL}/live/{camera_id}.flv"
    camera_stats[camera_id] = {'frames': 0, 'sent': 0, 'fps': 0}
    fps_times = deque(maxlen=30)
    
    # Connect WebSocket
    ws = TrafficWebSocket(BACKEND_WS_URL, camera_id, camera_keys.get(camera_id, ''))
    if ws.connect():
        log(f'✅ [{camera_id[-4:]}] WS Connected')
        camera_ws[camera_id] = ws
    else:
        log(f'❌ [{camera_id[-4:]}] WS Failed')
        return
    
    frame_interval = 1.0 / MAX_FPS if MAX_FPS > 0 else 0
    last_time = 0
    
    while True:
        try:
            cap = cv2.VideoCapture(flv_url)
            if not cap.isOpened(): time.sleep(2); continue
            log(f'🎥 [{camera_id[-4:]}] Stream connected')
            
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret: break
                
                now = time.time()
                if frame_interval > 0 and (now - last_time) < frame_interval:
                    continue  # Skip frame for FPS limiting
                last_time = now
                
                # Detect
                result = detect_frame_fast(frame, camera_id)
                
                # Send
                if ws.send(result):
                    camera_stats[camera_id]['sent'] += 1
                
                # FPS calc
                fps_times.append(now)
                if len(fps_times) > 1:
                    fps = len(fps_times) / (fps_times[-1] - fps_times[0])
                    camera_stats[camera_id]['fps'] = round(fps, 1)
                
                camera_stats[camera_id]['frames'] += 1
                
                # Log every 100 frames
                if camera_stats[camera_id]['frames'] % 100 == 0:
                    inf_time = result['vehicle'].get('inference_time', 0)
                    log(f"[{camera_id[-4:]}] {camera_stats[camera_id]['fps']} FPS | {inf_time:.0f}ms | {len(result['vehicle']['detections'])} det")
            
            cap.release()
        except Exception as e:
            log(f'❌ [{camera_id[-4:]}] {e}')
            time.sleep(2)

log('🔍 Fetching cameras...')
cameras = fetch_cameras()
if not cameras:
    log('❌ No cameras!')
else:
    log(f'✅ {len(cameras)} cameras')
    threads = []
    for cid in cameras:
        t = threading.Thread(target=process_camera, args=(cid,), daemon=True)
        t.start(); threads.append(t)
    log('🚀 Running MAX SPEED!')
    for t in threads: t.join()